In [7]:
# AutoDL官方学术资源加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [8]:
import sys
import os

# 添加项目根目录到Python路径
project_root = "/home/cuipeng/Gemma"
sys.path.append(project_root)

# 现在可以正常导入src下的模块
from src.core.model.model_initializer import initialize_model_and_tokenizer
from src.core.utils.model_utils import generate_response, apply_chat_template

In [9]:
import os
import json
import numpy as np
from datasets import load_dataset
import pandas as pd

In [10]:
class DomainSpecificBaseDatasetBuilder:
    def __init__(self):
        """初始化专业领域数据集构建器"""
        self.datasets = {}
        self.processed_data = []
        
    def load_task_datasets(self):
        """加载专业领域相关的数据集"""
        print("正在加载专业领域数据集...")
        
        try:
            # 中国现代诗词数据集 - 246568行
            self.datasets['modern_poetry'] = load_dataset('Iess/chinese_modern_poetry', split='train')
            print("✓ 中国现代诗词数据集加载完成") # https://huggingface.co/datasets/Iess/chinese_modern_poetry

            # 中国古代诗词数据集 - 193808行
            self.datasets['ancient_poetry'] = load_dataset('ddnoodle/chinese_poetry', split='train')
            print("✓ 中国古代诗词数据集加载完成") # https://huggingface.co/datasets/ddnoodle/chinese_poetry
            
            # 历史文献数据集 - 文言文，中国历史数据文言文比较多，增强文言文的理解能力 - 765,294行
            self.datasets['historical_documents'] = load_dataset('HistoryTrans/Dataset', split='train')
            print("✓ 历史文献数据集加载完成") # https://huggingface.co/datasets/HistoryTrans/Dataset
            
        except Exception as e:
            print(f"加载数据集时出错: {e}")
            raise
    
    def preprocess_data(self, max_samples_per_task=15000):
        """预处理各个任务数据集"""
        for dataset_name, dataset in self.datasets.items():
            print(f"处理 {dataset_name} 数据集...")
            
            if dataset_name == 'modern_poetry':
                processed = self._process_modern_poetry(dataset, max_samples_per_task)
            elif dataset_name == 'ancient_poetry':
                processed = self._process_ancient_poetry(dataset, max_samples_per_task)
            elif dataset_name == 'historical_documents':
                processed = self._process_historical(dataset, max_samples_per_task)
                
            self.processed_data.extend(processed)
            print(f"✓ {dataset_name} 处理完成，添加 {len(processed)} 条数据")
    
    def _process_modern_poetry(self, dataset, max_samples):
        """处理现代诗词数据集"""
        dataset = dataset[:3*max_samples]
        processed = []
        # print(type(dataset)) # <class 'dict'>
        # print(dataset.keys(), type(dataset["prompt"]), type(dataset["response"][0])) # dict_keys(['uuid', 'prompt', 'response']) <class 'list'> <class 'str'>

        # 处理所有数据
        for i in range(len(dataset['uuid'])):
            # print(type(item), item) # <class 'str'> uuid - 哈？着什么玩意
            # 创作任务
            processed.append({
                'messages': [
                    {"role": "user", "content": dataset['prompt'][i]},
                    {"role": "model", "content": dataset['response'][i]}
                ],
                'type': 'modern_poetry_creation'
            })
        
        print(f"现代诗词数据总量: {len(processed)} 条")
        
        # 如果数据量超过max_samples，随机采样
        if len(processed) > max_samples:
            processed = np.random.choice(processed, max_samples, replace=False).tolist()
            print(f"采样后数据量: {len(processed)} 条")
        
        return processed
    
    def _process_ancient_poetry(self, dataset, max_samples):
        """处理古代诗词数据集"""
        dataset = dataset[:3*max_samples]
        processed = []
        
        # 处理所有数据
        for i in range(len(dataset['instruction'])):
            # 创作任务
            processed.append({
                'messages': [
                    {"role": "user", "content": f"请根据以下题目：{dataset['context'][i]}，{dataset['instruction'][i]}。"},
                    {"role": "model", "content": dataset['response'][i]}
                ],
                'type': 'ancient_poetry_creation'
            })
        
        print(f"古代诗词数据总量: {len(processed)} 条")
        
        # 如果数据量超过max_samples，随机采样
        if len(processed) > max_samples:
            processed = np.random.choice(processed, max_samples, replace=False).tolist()
            print(f"采样后数据量: {len(processed)} 条")
        
        return processed
    
    def _process_historical(self, dataset, max_samples):
        """处理历史文献数据集"""
        dataset = dataset[:3*max_samples]
        processed = []
        
        # 处理所有数据
        for i in range(len(dataset['inputs'])):
            # 文言文翻译
            processed.append({
                'messages': [
                    {"role": "user", "content": f"请将以下文言文翻译成现代文：\n\n{dataset['inputs'][i]}"},
                    {"role": "model", "content": dataset['truth'][i]}
                ],
                'type': 'classical_translation'
            })
        
        print(f"历史文献数据总量: {len(processed)} 条")
        
        # 如果数据量超过max_samples，随机采样
        if len(processed) > max_samples:
            processed = np.random.choice(processed, max_samples, replace=False).tolist()
            print(f"采样后数据量: {len(processed)} 条")
        
        return processed
    
    def format_for_training(self):
        """将数据格式化为训练所需的格式"""
        formatted_data = []
        
        for item in self.processed_data:
            dialogue_str = apply_chat_template(item['messages'])
            formatted_data.append({
                'text': dialogue_str,
                'type': item['type']  # 保持原始任务类型
            })
            
        return formatted_data
    
    def save_dataset(self, formatted_data, output_path = "stage3/data_raw", train_ratio=0.95, train_size=30000, valid_size=1000):
        """保存数据集为训练集和验证集
        
        Args:
            formatted_data: 格式化后的数据
            train_ratio: 训练集比例
        """
        # 创建输出目录
        os.makedirs(output_path, exist_ok=True)
        
        # 随机打乱数据
        np.random.shuffle(formatted_data)
        
        # 分割训练集和验证集
        split_idx = int(len(formatted_data) * train_ratio)
        train_data = formatted_data[:split_idx]
        valid_data = formatted_data[split_idx:]

        # 从训练集中随机选择指定数量的数据
        if len(train_data) > train_size:
            train_indices = np.random.choice(len(train_data), train_size, replace=False)
            train_data = [train_data[i] for i in train_indices]

        # 从验证集中随机选择指定数量的数据
        if len(valid_data) > valid_size:
            valid_indices = np.random.choice(len(valid_data), valid_size, replace=False)
            valid_data = [valid_data[i] for i in valid_indices]
        
        # 保存数据集
        train_path = os.path.join(output_path, 'train.json')
        valid_path = os.path.join(output_path, 'valid.json')
        
        with open(train_path, 'w', encoding='utf-8') as f:
            json.dump(train_data, f, ensure_ascii=False, indent=2)
        
        with open(valid_path, 'w', encoding='utf-8') as f:
            json.dump(valid_data, f, ensure_ascii=False, indent=2)
        
        print(f"\n数据集已保存：")
        print(f"- 训练集: {train_path} ({len(train_data)} 条数据)")
        print(f"- 验证集: {valid_path} ({len(valid_data)} 条数据)")
        
        # 输出数据集统计信息
        self._print_dataset_stats(formatted_data)
    
    def _print_dataset_stats(self, data):
        """打印数据集统计信息"""
        print("\n数据集统计信息:")
        
        # 统计不同类型的数据数量
        type_counts = {}
        for item in data:
            type_counts[item['type']] = type_counts.get(item['type'], 0) + 1
            
        print("\n数据类型分布:")
        for data_type, count in type_counts.items():
            percentage = (count / len(data)) * 100
            print(f"- {data_type}: {count} ({percentage:.2f}%)")

In [11]:
def main():
    # 1. 初始化数据集构建器
    builder = DomainSpecificBaseDatasetBuilder()
    
    try:
        # 2. 加载数据集
        builder.load_task_datasets()
        
        # 3. 处理数据集
        builder.preprocess_data(max_samples_per_task=20000)
        
        # 4. 格式化数据
        formatted_data = builder.format_for_training()
        
        # 5. 保存数据集
        builder.save_dataset(formatted_data)
        
        print("\n专业领域阶段基础数据集构建完成！")
        
    except Exception as e:
        print(f"构建数据集时出错: {e}")
        raise

In [12]:
if __name__ == "__main__":
    main()

正在加载专业领域数据集...
✓ 中国现代诗词数据集加载完成
✓ 中国古代诗词数据集加载完成
✓ 历史文献数据集加载完成
处理 modern_poetry 数据集...
现代诗词数据总量: 60000 条
采样后数据量: 20000 条
✓ modern_poetry 处理完成，添加 20000 条数据
处理 ancient_poetry 数据集...
古代诗词数据总量: 60000 条
采样后数据量: 20000 条
✓ ancient_poetry 处理完成，添加 20000 条数据
处理 historical_documents 数据集...
历史文献数据总量: 60000 条
采样后数据量: 20000 条
✓ historical_documents 处理完成，添加 20000 条数据

数据集已保存：
- 训练集: stage3/data_raw/train.json (30000 条数据)
- 验证集: stage3/data_raw/valid.json (1000 条数据)

数据集统计信息:

数据类型分布:
- modern_poetry_creation: 20000 (33.33%)
- classical_translation: 20000 (33.33%)
- ancient_poetry_creation: 20000 (33.33%)

专业领域阶段基础数据集构建完成！
